In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
import os

if not os.path.exists('/content/drive/MyDrive/Group project/cleaned_data'):
    os.makedirs('/content/drive/MyDrive/Group project/cleaned_data')

In [24]:
df_restaurants = pd.read_csv('/content/drive/MyDrive/Group project/data/restaurants.csv')
df_restaurants.head()

,id,name,type_of_food,address,short_address,location,district,lat,lng,national_phone,...,accessibility_options,parking_options,payment_options,takeout,delivery,dine_in,google_maps_uri,current_popularity,populartimes,time_wait
0,ChIJOaDRwjtRqEcRnkLvxNMuZdw,Coréen Restaurant Berlin,Korean,"Torstraße 179, 10115 Berlin, Germany","Torstraße 179, Berlin","{'lat': 52.5287438, 'lng': 13.3939059}",Mitte,52.528744,13.393906,030 28883888,...,"{'wheelchairAccessibleParking': False, 'wheelc...","{'freeParkingLot': False, 'paidParkingLot': Tr...","{'acceptsCreditCards': True, 'acceptsDebitCard...",True,False,True,https://maps.google.com/?cid=15881151148068913...,75.0,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",NaN
1,ChIJs-hQDcRRqEcR3HX3Tir_5M0,Mmaah - Eat Korean!,Korean,"Elisabeth-Schwarzhaupt-Platz 5, 10115 Berlin, ...","Elisabeth-Schwarzhaupt-Platz 5, Berlin","{'lat': 52.5325433, 'lng': 13.3879003}",Mitte,52.532543,13.387900,0176 75824086,...,{'wheelchairAccessibleEntrance': True},NaN,"{'acceptsCreditCards': True, 'acceptsDebitCard...",True,True,True,https://maps.google.com/?cid=14836263629643806...,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",NaN
2,ChIJ7Y1MwOZRqEcRqYdK1VDzYx4,soopoollim,Korean,"Ackerstraße 149, 10115 Berlin, Germany","Ackerstraße 149, Berlin","{'lat': 52.5315761, 'lng': 13.3961477}",Mitte,52.531576,13.396148,NaN,...,NaN,{'paidStreetParking': True},"{'acceptsCreditCards': True, 'acceptsDebitCard...",True,NaN,True,https://maps.google.com/?cid=21898613723335576...,NaN,NaN,NaN
3,ChIJMferFeJRqEcR4ILthWXklLA,YamYam Berlin,Korean,"Alte Schönhauser Str. 6, 10119 Berlin, Germany","Alte Schönhauser Str. 6, Berlin","{'lat': 52.5275836, 'lng': 13.4082387}",Mitte,52.527584,13.408239,030 24632485,...,"{'wheelchairAccessibleParking': False, 'wheelc...",{'paidStreetParking': True},"{'acceptsCreditCards': False, 'acceptsDebitCar...",True,True,True,https://maps.google.com/?cid=12724045971918258...,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,..."
4,ChIJB39gGgBRqEcR81n8a7nl0fY,Gippeum,Korean,"Chausseestraße 122, 10115 Berlin, Germany","Chausseestraße 122, Berlin","{'lat': 52.52920899999999, 'lng': 13.3841791}",Mitte,52.529209,13.384179,NaN,...,NaN,{'paidStreetParking': True},NaN,True,NaN,True,https://maps.google.com/?cid=17785248988006210...,NaN,NaN,NaN


In [25]:
df_restaurants.shape

(3434, 30)

In [26]:
# Drop unnecessary columns
df_restaurants.drop(columns=["id", "address", "short_address", "location",
                             "national_phone","international_phone", "website",
                             "business_status", "current_opening_hours",
                             "editorial_summary", "google_maps_uri",
                             "current_popularity", "payment_options",
                             "time_wait", "types", "regular_opening_hours",
                             "lat", "lng"],
                    inplace=True, errors='ignore')
df_restaurants.columns

Index(['name', 'type_of_food', 'district', 'rating', 'total_ratings',
       'price_level', 'accessibility_options', 'parking_options', 'takeout',
       'delivery', 'dine_in', 'populartimes'],
      dtype='object')

In [27]:
df_restaurants.isna().sum().sort_values(ascending=False)

,0
price_level,1285
populartimes,1031
parking_options,1012
accessibility_options,636
delivery,490
takeout,238
dine_in,68
rating,22
total_ratings,22
district,0


In [28]:
# Dealing with missing values
df_restaurants.dropna(subset=['rating', 'total_ratings'], inplace=True)

df_restaurants.fillna({"price_level": "NO_PRICE_DATA", "dine_in": "FALSE",
                      "takeout": "FALSE", "delivery": "FALSE"}, inplace=True)

In [29]:
def safe_json_loads(x):
    if pd.isna(x):
        return None

    try:
        x = (
            x.replace("'", '"')
             .replace("True", "true")
             .replace("False", "false")
             .replace("None", "null")
        )
        return json.loads(x)
    except json.JSONDecodeError:
        print("Invalid JSON:", x)
        return None

def pick(d, key):
    return d.get(key, False) if isinstance(d, dict) else False

In [30]:
if 'accessibility_options' in df_restaurants.columns:
    df_restaurants['accessibility_options_json'] = (
        df_restaurants['accessibility_options'].apply(safe_json_loads)
    )

    df_restaurants['wheelchair_accessible_entrance'] = df_restaurants['accessibility_options_json'].apply(lambda d: pick(d, 'wheelchairAccessibleEntrance'))

    df_restaurants.drop(columns=['accessibility_options', 'accessibility_options_json'],
                        inplace=True, errors='ignore')

In [31]:
if 'parking_options' in df_restaurants.columns:
    df_restaurants['parking_options_json'] = (
        df_restaurants['parking_options'].apply(safe_json_loads)
    )

    df_restaurants['free_street_parking'] = df_restaurants['parking_options_json'].apply(lambda d: pick(d, 'freeStreetParking'))

    df_restaurants.drop(columns=['parking_options', 'parking_options_json'],
                        inplace=True, errors='ignore')

In [32]:
DISTRICT = "Lichtenberg"
TYPE_OF_FOOD = "Mexican"

In [33]:
df_restaurants_filtered = df_restaurants[
    (df_restaurants['type_of_food'] == TYPE_OF_FOOD)
]

df_restaurants_filtered.head()

,name,type_of_food,district,rating,total_ratings,price_level,takeout,delivery,dine_in,populartimes,wheelchair_accessible_entrance,free_street_parking
164,Cantina Mexicana Que Pasa | Mexikanisches Rest...,Mexican,Mitte,4.7,6393.0,PRICE_LEVEL_MODERATE,True,False,True,"[{'name': 'Monday', 'data': [20, 0, 0, 0, 0, 0...",False,False
165,El Amigo Taqueria,Mexican,Mitte,4.8,1358.0,NO_PRICE_DATA,True,False,True,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",True,False
166,Taco Loco Cantina Mexicana,Mexican,Mitte,4.4,506.0,PRICE_LEVEL_MODERATE,True,True,True,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",False,False
167,Tex-Mex,Mexican,Mitte,4.3,2064.0,PRICE_LEVEL_MODERATE,True,False,True,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",False,False
168,El Zarape | Mexican Food & Cocktail bar,Mexican,Mitte,4.7,534.0,NO_PRICE_DATA,True,True,True,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",False,False


In [34]:
# Initialize lists to store the data
days_list = []
hours_list = []
values_list = []

In [35]:
# Iterate through each restaurant's populartimes
for populartimes in df_restaurants_filtered['populartimes']:
    parsed_data = safe_json_loads(populartimes)

    if parsed_data is not None:
        # Extract data for each day
        for day_data in parsed_data:
            day_name = pick(day_data, 'name')
            hourly_data = pick(day_data, 'data')

            if day_name and hourly_data:
                # Add each hour's data
                for hour, value in enumerate(hourly_data):
                    days_list.append(day_name)
                    hours_list.append(hour)
                    values_list.append(value)

In [36]:
# Create the dataframe
df_populartimes_temp = pd.DataFrame({
    'day': days_list,
    'hour': hours_list,
    'value': values_list
})

In [37]:
df_populartimes = (
    df_populartimes_temp
    .groupby(['day', 'hour'])['value']
    .mean()
    .reset_index()
)

df_populartimes.columns = ['day', 'hour', 'value']

In [38]:
df_populartimes.head()

,day,hour,value
0,Friday,0,2.250000
1,Friday,1,0.294643
2,Friday,2,0.116071
3,Friday,3,0.000000
4,Friday,4,0.000000


In [39]:
df_populartimes.shape

(168, 3)

In [40]:
df_populartimes.dtypes

,0
day,object
hour,int64
value,float64


In [41]:
df_populartimes.to_csv('/content/drive/MyDrive/Group project/cleaned_data/cleaned_populartimes.csv', index=False, encoding="utf-8-sig")

In [42]:
df_restaurants.drop(columns=['populartimes'], inplace=True, errors='ignore')

In [43]:
# Typecheck
df_restaurants.dtypes

,0
name,object
type_of_food,object
district,object
rating,float64
total_ratings,float64
price_level,object
takeout,object
delivery,object
dine_in,object
wheelchair_accessible_entrance,bool


In [44]:
df_restaurants['total_ratings'] = df_restaurants['total_ratings'].astype(int)
df_restaurants['dine_in'] = df_restaurants['dine_in'].astype(bool)
df_restaurants['takeout'] = df_restaurants['takeout'].astype(bool)
df_restaurants['delivery'] = df_restaurants['delivery'].astype(bool)

In [45]:
df_restaurants.dtypes

,0
name,object
type_of_food,object
district,object
rating,float64
total_ratings,int64
price_level,object
takeout,bool
delivery,bool
dine_in,bool
wheelchair_accessible_entrance,bool


In [46]:
df_district = pd.read_csv('/content/drive/MyDrive/Group project/data/berlin_district_info.csv')
df_district

,berlin_district,population,area_in_km,pop_density,lat,lng
0,Mitte,364.530,3940,10.080,52.5200,13.4050
1,Friedrichshain-Kreuzberg,266.583,2040,14.385,52.5090,13.4310
2,Pankow,409.453,10322,4.111,52.5667,13.4167
3,Charlottenburg-Wilmersdorf,323.507,6469,5.303,52.5078,13.3039
4,Spandau,246.257,9188,2.798,52.5511,13.1997
5,Steglitz-Zehlendorf,295.786,10256,3.027,52.4415,13.2610
6,Tempelhof-Schöneberg,337.361,5305,6.708,52.4669,13.3500
7,Neukölln,307.113,4493,7.345,52.4800,13.4350
8,Treptow-Köpenick,288.802,16773,1.753,52.4400,13.5800
9,Marzahn-Hellersdorf,282.847,6182,4.723,52.5400,13.5750


In [47]:
df_district.rename(columns={'pop_density': 'population_density', 'berlin_district': 'district'}, inplace=True)

In [48]:
df_district.dtypes

,0
district,object
population,float64
area_in_km,int64
population_density,float64
lat,float64
lng,float64


In [49]:
df_restaurants_district_info = pd.merge(df_restaurants, df_district[['district', 'population_density']], on='district', how='left')

In [50]:
df_restaurants_district_info.head()

,name,type_of_food,district,rating,total_ratings,price_level,takeout,delivery,dine_in,wheelchair_accessible_entrance,free_street_parking,population_density
0,Coréen Restaurant Berlin,Korean,Mitte,4.2,488,PRICE_LEVEL_MODERATE,True,False,True,False,False,10.08
1,Mmaah - Eat Korean!,Korean,Mitte,4.7,966,NO_PRICE_DATA,True,True,True,True,False,10.08
2,soopoollim,Korean,Mitte,4.8,412,NO_PRICE_DATA,True,True,True,False,False,10.08
3,YamYam Berlin,Korean,Mitte,4.3,3468,PRICE_LEVEL_MODERATE,True,True,True,False,False,10.08
4,Gippeum,Korean,Mitte,4.7,117,NO_PRICE_DATA,True,True,True,False,False,10.08


In [51]:
df_restaurants_district_info.to_csv('/content/drive/MyDrive/Group project/cleaned_data/cleaned_restaurants_and_district_info.csv', index=False, encoding="utf-8-sig")

In [52]:
df_christmas_market = pd.read_csv('/content/drive/MyDrive/Group project/data/christmas_market.csv')
df_christmas_market.head()

,id,bezirk,name,strasse,plz,von,bis,veranstalter,oeffnungszeiten,email,w3,bemerkungen
0,58,Treptow-Köpenick,Winterzauber Weihnachtsmarkt,Möllhausenufer 30,12555,01.11.2024,31.12.2024,Strandbad Wendenschloss GmbH,12:00 - 20:00 Uhr,info@strandbad-wendenschloss.berlin,https://strandbad-wendenschloss.berlin,/
1,181,Mitte,Smash Burger,"Landsberger Allee, Rotes Rathaus, Alexander Platz",10178,01.11.2024,28.12.2024,Mr. Smash Burger,14:00 Uhr bis 23:00 Uhr,vivien-selimi@gmx.de,/,/
2,214,Mitte,Weddingmarkt,Leopoldplatz,13353,01.12. / 08.12. / 15.12. / 22.12.2024,NaN,Weddingmarkt - Sabrina Pützer,11:00 Uhr bis 19:00 Uhr,weddingkulturmarkt@gmail.com,https://www.wedding-markt.de/,Weddingmarkt wird Weihnachtsmarkt. An allen 4....
3,61,Steglitz-Zehlendorf,Kunsthandwerklicher Weihnachtsmarkt Mexikoplatz,Mexikoplatz und Bülowstraße 1-8,14163,01.12. / 15.12. / 22.12.2024,NaN,KUNSTHAND-BERLIN Cornelja Hasler,11:00 - 18:00 Uhr,info@kunsthand-berlin.de,https://kunsthand-berlin.de/,Es werden nur handgefertigte Objekte kleiner A...
4,97,Mitte,Adventsbasar an der Heilandskirche,Thusnelda-Allee 1,10555,01.12.2024,01.12.2024,Marktzeit - Brigitta Voigt,12:00 -18:00 Uhr,marktzeit@posteo.de,https://www.marktzeit.berlin/,Adventsmarkt findet nur am 1. Advent statt.


In [53]:
df_christmas_market['bezirk'].value_counts()

,count
bezirk,
Steglitz-Zehlendorf,12
Charlottenburg-Wilmersdorf,11
Mitte,9
Brandenburg,8
Tempelhof-Schöneberg,7
Pankow,7
Treptow-Köpenick,6
Marzahn-Hellersdorf,4
Reinickendorf,3


In [54]:
df_christmas_market.to_csv('/content/drive/MyDrive/Group project/cleaned_data/cleaned_christmas_market.csv', index=False, encoding="utf-8-sig")

In [55]:
df_street_food_and_folk_festivals = pd.read_csv('/content/drive/MyDrive/Group project/data/street_food_and_folk_festivals.csv')
df_street_food_and_folk_festivals.head()

,id,bezirk,bezeichnung,strasse,plz,von,bis,zeit,veranstalter,mail,www,barrierefreiheit,bemerkungen
0,19,Tempelhof-Schöneberg,650 Jahre Lichtenrade rund um den Lichtenrader...,Alt-Lichtenrade,12309.0,01.05.2025,04.05.2025,Do/Fr/Sa 12:00 bis 23:00 Uhr \nSo 12:00 bis 20...,Family &amp; Friends e. V.,stadtkultur@family-and-friends-ev.de,https://www.family-and-friends-ev.de/1-4-mai-2...,"Ja, findet auf der Straße statt",Wo kann man 650 Jahre Lichtenrade besser feier...
1,16,Mitte,30 Jahre UMWELTFESTIVAL – jetzt erst recht,"Straße des 17. Juni, am Brandenburger Tor",10117.0,01.06.2025,NaN,11:00 bis 18:00 Uhr,GRÜNE LIGA Berlin e.V,info@umweltfestival.de,https://www.umweltfestival.de/,Ja,Das ganze Veranstaltungsgelände ist ebenerdig ...
2,142,Reinickendorf,Gorkistraßenfest Tegel,Gorkistraße 11/21,13507.0,01.08.2025,02.08.2025,Fr 15-22 Uhr \nSa 12-22 Uhr,Hauptstadt Kultur,info@juettner-entertainment.de,https://www.hauptstadtkultur.berlin,Ja,Mit dem Gorkistraßenfest wird eine neue Verans...
3,73,Marzahn-Hellersdorf,Biesdorfer Blütenfest,"Schlosspark Biesdorf, Alt-Biesdorf 55",12683.0,02.05.2025,04.05.2025,Fr 16-22 Uhr Sa 12-22 Uhr So 12-18 Uhr,Hauptstadt Kultur,info@juettner-entertainment.de,https://www.hauptstadtkultur.berlin,Ja,Neben dem musikalischen Rahmenprogramm erwarte...
4,169,Treptow-Köpenick,Köpenicker Herbst,Luisenhain Köpenick,12555.0,02.10.2025,05.10.2025,Fr 12-22 Uhr Sa 12-22 Uhr So 12-20 Uhr,Hauptstadt Kultur,info@juettner-entertainment.de,https://www.hauptstadtkultur.berlin,Ja,Spaß für Jung und Alt mit Spiel und Fahrgeschä...


In [56]:
#Drop unnecessary columns
df_street_food_and_folk_festivals.drop(columns=['id', 'bezeichnung', 'strasse', 'plz', 'veranstalter',	'mail',	'www', 'bemerkungen'], inplace=True)
df_street_food_and_folk_festivals.head()

,bezirk,von,bis,zeit,barrierefreiheit
0,Tempelhof-Schöneberg,01.05.2025,04.05.2025,Do/Fr/Sa 12:00 bis 23:00 Uhr \nSo 12:00 bis 20...,"Ja, findet auf der Straße statt"
1,Mitte,01.06.2025,NaN,11:00 bis 18:00 Uhr,Ja
2,Reinickendorf,01.08.2025,02.08.2025,Fr 15-22 Uhr \nSa 12-22 Uhr,Ja
3,Marzahn-Hellersdorf,02.05.2025,04.05.2025,Fr 16-22 Uhr Sa 12-22 Uhr So 12-18 Uhr,Ja
4,Treptow-Köpenick,02.10.2025,05.10.2025,Fr 12-22 Uhr Sa 12-22 Uhr So 12-20 Uhr,Ja


In [57]:
#change column names
df_street_food_and_folk_festivals.columns = ["district", "from", "to", "time", "entrance"]
df_street_food_and_folk_festivals['district'].value_counts()

,count
district,
Reinickendorf,12
Tempelhof-Schöneberg,8
Treptow-Köpenick,8
Charlottenburg-Wilmersdorf,6
Steglitz-Zehlendorf,6
Pankow,6
Lichtenberg,5
Mitte,4
Marzahn-Hellersdorf,4


In [58]:
df_street_food_and_folk_festivals.to_csv('/content/drive/MyDrive/Group project/cleaned_data/cleaned_street_food_and_folk_festivals.csv', index=False, encoding="utf-8-sig")

In [59]:
df_companies = pd.read_csv('/content/drive/MyDrive/Group project/data/companies.csv')
df_companies.head()

,id,name,district
0,ChIJLyG1ae5RqEcRV7CDd2ppmHA,Afilio GmbH,Mitte
1,ChIJIb_c4-tRqEcRXYbvOFkDjYw,IBM iX Berlin GmbH,Mitte
2,ChIJNZGrWOxRqEcR1cy2vmMmUsU,HERE Berlin,Mitte
3,ChIJl3hWxLlRqEcRtwCZURdqxFo,Everstores Technologies GmbH HQ,Mitte
4,ChIJO23u5jdOqEcRHiuo9oNkDBk,Sensorberg GmbH,Mitte


In [60]:
df_companies_grouped_by_district = df_companies.groupby('district').size().reset_index(name='count')
df_companies_grouped_by_district.sort_values(by='count', ascending=False)

,district,count
4,Mitte,1568
0,Charlottenburg-Wilmersdorf,1344
6,Pankow,1000
9,Steglitz-Zehlendorf,900
10,Tempelhof-Schöneberg,846
1,Friedrichshain-Kreuzberg,748
11,Treptow-Köpenick,719
5,Neukölln,657
7,Reinickendorf,504
3,Marzahn-Hellersdorf,465


In [61]:
df_companies_grouped_by_district.to_csv('/content/drive/MyDrive/Group project/cleaned_data/cleaned_companies_grouped_by_district.csv', index=False, encoding="utf-8-sig")

In [62]:
df_restaurants_district_info.describe()

,rating,total_ratings,population_density
count,3412.000000,3412.000000,3412.000000
mean,4.496805,842.134525,6.516986
std,0.348955,1589.535931,3.636017
min,1.000000,1.000000,1.753000
25%,4.400000,181.000000,3.027000
50%,4.500000,468.500000,5.303000
75%,4.700000,946.000000,10.080000
max,5.000000,36921.000000,14.385000
